# This notebook is for running any huggingface model

## Installing needed libraries

In [ ]:
 !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

## Importing needed libraries

In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig, Trainer, pipeline
from peft import LoraConfig
from datasets import Dataset
from langchain.prompts.prompt import PromptTemplate

#from trl import SFTTrainer
from peft import PeftModel

import pandas as pd

### My Finetuned Model

In [ ]:
model_name="Nishika26/llama2-sql-sft-merged"

In [ ]:
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
                 model_name,
                 quantization_config=bnb_config,
                 resume_download=True
                 )
pipe = pipeline("text-generation",
               model=model,
               tokenizer=tokenizer,
               max_new_tokens=512
               )

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [ ]:
prompt_template = PromptTemplate(
    input_variables=["instruction", "category", "schema", "question"], template="<s>[INST]<<SYS>>You are an agent designed to interact with a SQL database.Given an input question, create a syntactically correct teradatasql query to run. This is a {category} question and here is the {ddl} that should be considered will answering the question.</SYS>{question}[/INST]"
)

# what is the category of the question
category = """Supplier Payments"""


#put your query
question = """Do I have any payments which I need to make which are delayed?"""

#data defination language of the database table being considered
ddl="""CREATE TABLE ap_payments (
	ap_payment_id serial4 NOT NULL,
	supplier_name varchar NULL,
	currency_code varchar NULL,
	payment_check_number varchar NULL,
	payment_amount numeric NULL,
	payment_date date NULL
);"""


input = prompt_template.format(category=category, ddl=ddl, question=question)

In [ ]:
input

In [ ]:
result = pipe(input)

In [ ]:
print(result[0]['generated_text'].split("/INST")[1][1:])

## Another approach

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 8.6 MB/s eta 0:00:00


In [ ]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

In [ ]:
hf = HuggingFacePipeline(pipeline=pipe)

query = "How may suppliers have Immediate payment terms?"

prompt = f"""
### System:
You are an agent designed to interact with a SQL database.Given an input query, create a syntactically correct teradatasql query to run. Give clear and technical answer, don't add any extra gibberish. Here is the schema of the database that you need to consider will answering the question :- "
CREATE TABLE ap_payments (
	ap_payment_id serial4 NOT NULL,
	supplier_name varchar NULL,
	currency_code varchar NULL,
	payment_check_number varchar NULL,
	payment_amount numeric NULL,
	payment_date date NULL
);"""

### User:
{query}

### Response:


response = hf.predict(prompt)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



### System:
You are an agent designed to interact with a SQL database.Given an input query, create a syntactically correct teradatasql query to run. Give clear and technical answer, don't add any extra gibberish. Here is the schema of the database that you need to consider will answering the question :- "CREATE TABLE ap_invoices (
ap_invoice_id serial4 NOT NULL,
supplier_id numeric NULL,
supplier_address_id numeric NULL,
src_invoice_number varchar NULL,
currency_code varchar NOT NULL,
invoice_date date NULL,
invoice_type varchar NULL,
invoice_source varchar NULL,
exchange_rate numeric NULL,
exchange_rate_to_usd numeric NULL,
invoice_amount_total numeric NULL,
approved_amount_total numeric NULL,
invoice_balance_total numeric NULL,
payment_term varchar NULL,
payment_term_desc varchar NULL,
payment_sequence numeric NULL,
payment_due_date date NULL,
payment_amount_due numeric NULL,
payment_amount_balance numeric NULL,
business_entity_id numeric NULL,
business_entity_name varchar NULL,
src